<!--NOTEBOOK_HEADER-->
*This notebook contains material from [CBE60499](https://ndcbe.github.io/CBE60499);
content is available [on Github](git@github.com:ndcbe/CBE60499.git).*


<!--NAVIGATION-->
< [2.5 Stochastic Programming](https://ndcbe.github.io/CBE60499/02.05-SP.html) | [Contents](toc.html) | [2.7 Pyomo Homework 2](https://ndcbe.github.io/CBE60499/02.07-Pyomo2.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.06-Pyomo1.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.06-Pyomo1.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

In [ ]:
# IMPORT DATA FILES USED BY THIS NOTEBOOK
import os,  requests

file_links = [("data/knapsack_data.csv", "https://ndcbe.github.io/CBE60499/data/knapsack_data.csv")]

# This cell has been added by nbpages. Run this cell to download data files required for this notebook.

for filepath, fileurl in file_links:
    stem, filename = os.path.split(filepath)
    if stem:
        if not os.path.exists(stem):
            os.mkdir(stem)
    if not os.path.isfile(filepath):
        with open(filepath, 'wb') as f:
            response = requests.get(fileurl)
            f.write(response.content)


# 2.6 Pyomo Homework 1

**Due Date:** 2/16/2021

In [ ]:
# This code cell installs packages on Colab

import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_idaes()
    helper.install_ipopt()
    helper.install_glpk()
    helper.download_data(['knapsack_data.xlsx','knapsack_data.csv'])

--2021-02-15 19:16:37--  https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6456 (6.3K) [text/plain]
Saving to: ‘helper.py’

helper.py           100%[===================>]   6.30K  --.-KB/s    in 0s      

2021-02-15 19:16:37 (79.0 MB/s) - ‘helper.py’ saved [6456/6456]

Installing idaes via pip...
idaes was successfully installed
Running idaes get-extensions to install Ipopt and k_aug...
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)

[K_AUG] written by D.T. @2018 (x86_64-pc-linux-gnu), ASL(20190605)

ipopt was successfully installed
k_aug was successfully installed
Installing glpk via apt-get...
Checking for ./data/knapsack_data.xlsx
Checking for ./data/knapsack_data.csv
	File found!


In [ ]:
## IMPORT LIBRARIES
from pyomo.environ import *
import pandas as pd

Special thanks to the Pyomo team for create these excercises as part of their excellent PyomoFest workshop.

## 2.6.1 Pyomo Fundamentals

### 2.6.1.1 Knapsack example

Solve the knapsack problem given below using GLPK and answer the following questions:

1. Which items are acquired in the optimal solution?

2. What is the value of the selected items?

In [ ]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14 # Weight max

model = ConcreteModel()
model.x = Var( A, within=Binary )

# Define parameters
model.benefits = Param(A, initialize = b)
model.weight = Param(A, initialize = w)

# Define objectives
model.obj = Objective(
    expr = sum( model.benefits[i]*model.x[i] for i in A ), 
    sense = maximize )
# Define constraints
model.weight_con = Constraint(
    expr = sum( model.weight[i]*model.x[i] for i in A ) <= W_max )

model.display()

# Solver
solver = SolverFactory('glpk')
results = solver.solve(model, tee= True)

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :  None :     1 : False :  True : Binary
        screwdriver :     0 :  None :     1 : False :  True : Binary
              towel :     0 :  None :     1 : False :  True : Binary
             wrench :     0 :  None :     1 : False :  True : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
ERROR: evaluating object as numeric value: x[hammer]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[hammer]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.SimpleObjective'>)
    No value for uninitialized NumericValue object x[hammer]
        Key : Active : Value
        None :   None :  None

  Constraints:
    weight_con : Size=1
ERROR: evaluating object as numeric value: x[hammer]
        (object: <class 'pyomo.c

In [ ]:
for i in A:
  print("Units of", i, "items =", model.x[i]())
print('The total value of items selected is', model.obj())
total_weight = sum( w[i]*value(model.x[i]) for i in A )
print('Total Weight of optimal solution', total_weight)

Units of hammer items = 1.0
Units of wrench items = 0.0
Units of screwdriver items = 1.0
Units of towel items = 1.0
The total value of items selected is 25.0
Total Weight of optimal solution 12.0


**Question Answers**

1. From the solution, the items acquired in the optimal solution includes hammer, screwdriver, and towel

2. We will need 1 hammer, 1 screwdriver and 1 towel and the total benefits values is 25 with optimal weight of 12

### 2.6.1.2 Knapsack example with improve printing

Complete the missing lines in the code below to produce formatted output: print the total weight, the value of the items selected (the objective), and the items acquired in the optimal solution. Note, the Pyomo value function should be used to get the floating point value of Pyomo modeling components (e.g., print(value(model.x[i])).

In [ ]:
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*value(model.x[i]) for i in A )
# YOUR SOLUTION HERE
print('Total Weight of optimal solution', total_weight)
print('The total value of items selected is', model.obj())
print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    acquired = 'No'
    if value(model.x[i]) >= 0.5:
        acquired = 'Yes'
    print('%12s %12s' % (i, acquired))
print('-------------------------')

Total Weight of optimal solution 12.0
The total value of items selected is 25.0
        Item     Selected
      hammer          Yes
      wrench           No
 screwdriver          Yes
       towel          Yes
-------------------------


### 2.6.1.3 Changing data

Using your code from **Question 1.2**, if we were to increase the value of the wrench, at what point would it become selected as part of the optimal solution?

In [ ]:
# YOUR SOLUTION HERE
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )
# For loop to increase the value of the wrench
opt = SolverFactory('glpk')
opt_success = opt.solve(model)

while model.x["wrench"]() == 0:
  # Change value of benefits
  b["wrench"] += 1
  # Update model.obj:
  model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )
  # Call solver
  opt = SolverFactory('glpk')
  opt_success = opt.solve(model)
  if model.x['wrench']() != 0:
    print('\nThe value of benefits for wrench to be selected in the optimal sol is', b["wrench"])

# Print all solutions
for i in A:
  if model.x[i] != 0:
    print('Items selected is', i, 'and its value is', model.x[i]())
total_weight = sum( w[i]*value(model.x[i]) for i in A )
print('Total Weight:', total_weight)
print('Total Benefit:', value(model.obj))

    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    'pyomo.core.base.objective.SimpleObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.SimpleObjecti

**Question Answer**

From the analysis, it can be seen that if benefits of wrench increases to 8, then it will be selected to be in the optimal solution. This basically replaces the selection of hammer which also has benefit of 8; the only difference here is the total weight by replacing hammer with wrench is higher with a value of 14 instead of 12.

### 2.6.1.4 Loading data from Excel

In the code above, the data is hardcoded at the top of the file. Instead of hardcoding the data, use Python to load the data from a difference source. You may use Pandas to load data from 'knapsack_data.xlsx' into a dataframe. You will then need to write code to obtain a dictionary from the dataframe.

In [ ]:
df_items = pd.read_excel('./data/knapsack_data.xlsx', sheet_name='data', header=0, index_col=0)
W_max = 14
#print(df_items.head())
A = df_items.index.tolist() # This will allow the index (e.g hammer) to be extracted
benefits = df_items.columns.to_list()[0] # Convert the first column values
weight = df_items.columns.to_list()[1] # Convert the second column values
b = df_items[benefits].to_dict() # Convert this to dictionary with key and value
w = df_items[weight].to_dict()

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

opt = SolverFactory('glpk')
opt_success = opt.solve(model)

total_weight = sum( w[i]*value(model.x[i]) for i in A )
print('Total Weight:', total_weight)
print('Total Benefit:', value(model.obj))

print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    acquired = 'No'
    if value(model.x[i]) >= 0.5:
        acquired = 'Yes'
    print('%12s %12s' % (i, acquired))
print('-------------------------')

Total Weight: 12.0
Total Benefit: 25.0
        Item     Selected
      hammer          Yes
      wrench           No
 screwdriver          Yes
       towel          Yes
-------------------------


### 2.6.1.5 NLP vs. MIP

Solve the knapsack problem with IPOPT instead of glpk. Print the solution values for model.x. What happened? Why?

*Hint*: Switch `glpk` to `ipopt` in the call to `SolverFactory`.

In [ ]:
from pyomo.environ import *

A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14

model = ConcreteModel()
model.x = Var( A, within=Binary )

model.obj = Objective(
    expr = sum( b[i]*model.x[i] for i in A ), 
    sense = maximize )

model.weight_con = Constraint(
    expr = sum( w[i]*model.x[i] for i in A ) <= W_max )

model.pprint()
# YOUR SOLUTION HERE
opt = SolverFactory('ipopt')
opt_success = opt.solve(model, tee=True)

print("\n")
for i in A:
    if value(model.x[i]) >= 0.5:
      print(model.x[i](), i, 'selected')
print('-------------------------')

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'hammer', 'wrench', 'screwdriver', 'towel'}

1 Var Declarations
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :  None :     1 : False :  True : Binary
        screwdriver :     0 :  None :     1 : False :  True : Binary
              towel :     0 :  None :     1 : False :  True : Binary
             wrench :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 8*x[hammer] + 3*x[wrench] + 6*x[screwdriver] + 11*x[towel]

1 Constraint Declarations
    weight_con : Size=1, Index=None, Active=True
        Key  : Lower : Body                                                      : Upper : Active
        None :  -Inf 

**Question Answers**
The same optimal solution is obtained, with 1 hammer, 1 screwdriver, and 1 towel are selected in the optimal solution. This is consistent with the solution produced by **glpk** and makes sense.

## 2.6.2 More Pyomo Fundamentals

### 2.6.2.1 Knapsack problem with rules

Rules are important for defining in-dexed constraints, however, they can also be used for single (i.e. scalar) constraints. Reimplement the knapsack model from **Question 1.1** using rules for the objective and the constraints.

In [ ]:
# YOUR SOLUTION HERE
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14 # Weight max

model = ConcreteModel()
model.x = Var( A, within=Binary )


# Define objectives
def benefits_rules(model):
  return sum(b[i]*model.x[i] for i in A )
model.obj = Objective(expr = benefits_rules(model),sense = maximize )

# Define constraints
def weight_rule(model):
  return sum(w[i]*model.x[i] for i in A) <= W_max # this is a shorter way to write a for loop
model.weight_con = Constraint(A, rule= weight_rule)

model.display()

# Solver
solver = SolverFactory('glpk')
results = solver.solve(model, tee= True)

# Print
print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
    acquired = 'No'
    if value(model.x[i]) >= 0.5:
        acquired = 'Yes'
    print('%12s %12s' % (i, acquired))
print('-------------------------')

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :  None :     1 : False :  True : Binary
        screwdriver :     0 :  None :     1 : False :  True : Binary
              towel :     0 :  None :     1 : False :  True : Binary
             wrench :     0 :  None :     1 : False :  True : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
ERROR: evaluating object as numeric value: x[hammer]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[hammer]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.SimpleObjective'>)
    No value for uninitialized NumericValue object x[hammer]
        Key : Active : Value
        None :   None :  None

  Constraints:
    weight_con : Size=4
ERROR: evaluating object as numeric value: x[hammer]
        (object: <class 'pyomo.c

### 2.6.2.2 Integer formulation of the knapsack problem

Consider again the knapsack problem. Assume now that we can acquire multiple items of the same type. In this new formulation, $x_i$ is now an integer variable instead of a binary variable. One way to formulate this problem is as follows:


$\max_{q,x} \sum_{i\in{A}}v_ix_i$

s.t. $\sum_{i\in{A}}w_ix_i\leq W_{max}$

$x_i=\sum_{j=0}^Njq_{i,j}, \forall i \in{A}$

$0\leq x\leq N$

$q_{i,j} \in {0,1}, \forall i \in A, j \in {0...N}$

Starting with your code from **Question 2.1**, implement this new formulation and solve. Is the solution surprising?

In [ ]:
# YOUR SOLUTION HERE
# YOUR SOLUTION HERE
A = ['hammer', 'wrench', 'screwdriver', 'towel']
b = {'hammer':8, 'wrench':3, 'screwdriver':6, 'towel':11}
w = {'hammer':5, 'wrench':7, 'screwdriver':4, 'towel':3}
W_max = 14 # Weight max

model = ConcreteModel()
N = 5
model.x = Var(A, within = Integers, bounds=(0,N)) # Update xi to be positive integers


# Define objectives
def benefits_rules(model):
  return sum(b[i]*model.x[i] for i in A )
model.obj = Objective(expr = benefits_rules(model),sense = maximize )

# Define constraints
def weight_rule(model):
  return sum(w[i]*model.x[i] for i in A) <= W_max # this is a shorter way to write a for loop
model.weight_con = Constraint(A, rule= weight_rule)

model.display()

# Solver
solver = SolverFactory('glpk')
results = solver.solve(model, tee= True)

# Print
print('%12s %12s' % ('Item', 'Selected'))
print('=========================')
for i in A:
  if model.x[i]() != 0:
    print('Items selected is', i, 'and its value is', model.x[i]())

print('Total Benefit:', value(model.obj))

Model unknown

  Variables:
    x : Size=4, Index=x_index
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
             hammer :     0 :  None :     5 : False :  True : Integers
        screwdriver :     0 :  None :     5 : False :  True : Integers
              towel :     0 :  None :     5 : False :  True : Integers
             wrench :     0 :  None :     5 : False :  True : Integers

  Objectives:
    obj : Size=1, Index=None, Active=True
ERROR: evaluating object as numeric value: x[hammer]
        (object: <class 'pyomo.core.base.var._GeneralVarData'>)
    No value for uninitialized NumericValue object x[hammer]
ERROR: evaluating object as numeric value: obj
        (object: <class 'pyomo.core.base.objective.SimpleObjective'>)
    No value for uninitialized NumericValue object x[hammer]
        Key : Active : Value
        None :   None :  None

  Constraints:
    weight_con : Size=4
ERROR: evaluating object as numeric value: x[hammer]
        (object: <class 

**Question Answer**

With the new conditions where x (number of items) is integer instead of binary and is bounded between 0 and N, new solutions with different number of items selected are produced. 

In the case of N equals to 1, which is similar to previous analysis, the maximum benefit is 25. If one set N equals to 2, the solver selects the combination of 2 screwdriver and 2 towels, giving the max benefit of 34. If one set N equals to 3, the solver selects the combination of 1 hammer and 3 towels, giving the max benefit to be 41. For N equals to 4, the solver returns 4 towels being selected and the maximum benefits is 44. Once N is larger than 4, the solver still returns 4 towels with maximum benefits of 44. This suggests that 44 is the maximum benefits can be achieved by any bound N of x items being selected.

In addition, one can see that the solution is not surprising, considering the fact that the benefit of towels is the highest while it has the lowest weight. It makes sense that as one can increase the number of items being selected up, selecting more towels would be in the optimal solution.

<!--NAVIGATION-->
< [2.5 Stochastic Programming](https://ndcbe.github.io/CBE60499/02.05-SP.html) | [Contents](toc.html) | [2.7 Pyomo Homework 2](https://ndcbe.github.io/CBE60499/02.07-Pyomo2.html) ><p><a href="https://colab.research.google.com/github/ndcbe/CBE60499/blob/master/docs/02.06-Pyomo1.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://ndcbe.github.io/CBE60499/02.06-Pyomo1.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>